In [1]:
!pip install -q pandas scikit-learn deep-translator tqdm transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00


In [3]:
import re
import json
import uuid
import time
import hashlib
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from deep_translator import GoogleTranslator as DeepGoogle

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries are imported.")

Libraries are imported.


# Configuration

In [11]:
SMALL_PATH = "mentalhealth.csv"
LARGE_PATH = "train.csv"

# Output directory
OUT_DIR = Path("output")
OUT_DIR.mkdir(exist_ok=True)

# Output files
MERGED_EN_PATH = OUT_DIR / "merged_en.csv"
MERGED_MULTILANG_PATH = OUT_DIR / "merged_multilang.csv"
MERGED_JSONL_PATH = OUT_DIR / "merged_multilang.jsonl"

# Translation settings
TARGET_LANGS = ["hi", "te"]  # Hindi, Telugu
SLEEP_BETWEEN_TRANSLATIONS = 0.3  # Seconds (rate limiting)
MAX_RETRIES = 5

# Cleaning settings
MIN_CHAR_LENGTH = 2
MAX_CHAR_LENGTH = 1000
REMOVE_URLS = False

print("Configuration loaded")
print(f"Input files: {SMALL_PATH}, {LARGE_PATH}")
print(f"Output directory: {OUT_DIR}")
print(f"Target languages: {TARGET_LANGS}")

Configuration loaded
Input files: mentalhealth.csv, train.csv
Output directory: output
Target languages: ['hi', 'te']


# Cleaning and Filtering

In [10]:
def clean_and_filter_dataset(df,
                             min_char_length=2,
                             max_char_length=1000,
                             remove_urls=False,
                             verbose=True):
    if verbose:
        print(f"CLEANING DATASET")
        print(f"Initial rows: {len(df)}")
        start = time.time()

    df_clean = df.copy()

    # Convert to string and basic normalization
    if verbose:
        print("  → Converting to string and normalizing...")

    df_clean['prompt'] = df_clean['prompt'].astype(str).str.strip()
    df_clean['response'] = df_clean['response'].astype(str).str.strip()

    # Fix Unicode artifacts
    replacements = {
        "\u00A0": " ",
        "Â": "",
        "â€™": "'",
        "â€œ": '"',
        "â€": '"',
        'â€"': "-",
    }

    for old, new in replacements.items():
        df_clean['prompt'] = df_clean['prompt'].str.replace(old, new, regex=False)
        df_clean['response'] = df_clean['response'].str.replace(old, new, regex=False)

    # Normalize multiple spaces/newlines to single space
    df_clean['prompt'] = df_clean['prompt'].str.replace(r'\s+', ' ', regex=True)
    df_clean['response'] = df_clean['response'].str.replace(r'\s+', ' ', regex=True)

    # Remove URLs
    if remove_urls:
        if verbose:
            print("  → Removing URLs...")
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        df_clean['prompt'] = df_clean['prompt'].str.replace(url_pattern, '', regex=True)
        df_clean['response'] = df_clean['response'].str.replace(url_pattern, '', regex=True)

        # Remove emails
        df_clean['prompt'] = df_clean['prompt'].str.replace(r'\S+@\S+', '', regex=True)
        df_clean['response'] = df_clean['response'].str.replace(r'\S+@\S+', '', regex=True)

        # Re-normalize after removal
        df_clean['prompt'] = df_clean['prompt'].str.strip()
        df_clean['response'] = df_clean['response'].str.strip()

    # Length-based filtering
    if verbose:
        print("  → Filtering by length...")

    initial = len(df_clean)

    # Calculate lengths once
    prompt_lens = df_clean['prompt'].str.len()
    response_lens = df_clean['response'].str.len()

    # Create mask for valid entries
    valid_mask = (
        (prompt_lens >= min_char_length) &
        (prompt_lens <= max_char_length) &
        (response_lens >= min_char_length) &
        (response_lens <= max_char_length) &
        (df_clean['prompt'] != 'nan') &      # Filter 'nan' strings
        (df_clean['response'] != 'nan') &
        (df_clean['prompt'] != 'None') &     # Filter 'None' strings
        (df_clean['response'] != 'None')
    )

    df_clean = df_clean[valid_mask].copy()

    # Reset index
    df_clean = df_clean.reset_index(drop=True)

    if verbose:
        elapsed = time.time() - start
        removed = initial - len(df_clean)
        pct_removed = (removed / initial * 100) if initial > 0 else 0
        pct_kept = ((len(df_clean) / len(df)) * 100) if len(df) > 0 else 0

        print(f"\nCLEANING COMPLETE!")
        print(f"Final rows: {len(df_clean)}")
        print(f"Removed: {removed} ({pct_removed:.1f}%)")
        print(f"Retained: {pct_kept:.1f}% of original")
        print(f"Time: {elapsed:.2f}s")

    return df_clean

print("Cleaning function defined")

Cleaning function defined


# Data Loading Functions

In [9]:
def stable_fingerprint(s, max_len=2000):
    if s is None:
        s = ""
    s = str(s).lower()[:max_len]
    return hashlib.md5(s.encode('utf-8')).hexdigest()

def make_id(prefix="u"):
    return f"{prefix}_{uuid.uuid4().hex[:12]}"

def load_small_dataset(path):
    p = Path(path)

    if p.suffix in ['.tsv', '.txt']:
        df = pd.read_csv(path, sep='\t', dtype=str, keep_default_na=False)
    else:
        try:
            df = pd.read_csv(path, dtype=str, keep_default_na=False)
        except:
            df = pd.read_csv(path, sep='\t', dtype=str, keep_default_na=False)

    # Detect columns
    qid_col = next((c for c in df.columns if c.lower().startswith('questionid')), None)
    q_col = next((c for c in df.columns if 'question' in c.lower() and 'id' not in c.lower()), None)
    a_col = next((c for c in df.columns if 'answer' in c.lower()), None)

    if not (q_col and a_col):
        raise ValueError(f"Couldn't detect question/answer columns in {path}. Columns: {list(df.columns)}")

    # Build standardized format
    rows = []
    for _, r in df.iterrows():
        qid = r[qid_col] if qid_col and qid_col in r.index else None
        q_txt = str(r[q_col]).strip()
        a_txt = str(r[a_col]).strip()

        rows.append({
            'id': f"faq_{qid}" if qid and str(qid).strip() else make_id("faq"),
            'source': p.name,
            'type': 'faq',
            'prompt': q_txt,
            'response': a_txt,
            'lang': 'en',
            'orig_id': qid
        })

    return pd.DataFrame(rows)

def load_large_dataset(path):
    p = Path(path)
    df = pd.read_csv(path, dtype=str, keep_default_na=False)

    # Detect columns
    ctx_col = next((c for c in df.columns if 'context' in c.lower()), None)
    resp_col = next((c for c in df.columns if 'response' in c.lower() or 'reply' in c.lower()), None)

    if not (ctx_col and resp_col):
        raise ValueError(f"Couldn't detect Context/Response columns in {path}. Columns: {list(df.columns)}")

    # Build standardized format
    rows = []
    for _, r in df.iterrows():
        ctx = str(r[ctx_col]).strip()
        resp = str(r[resp_col]).strip()
        fid = stable_fingerprint(ctx + resp)

        rows.append({
            'id': f"dlg_{fid}",
            'source': p.name,
            'type': 'dialogue',
            'prompt': ctx,
            'response': resp,
            'lang': 'en',
            'orig_id': None
        })

    return pd.DataFrame(rows)

print("Data loading functions defined")

Data loading functions defined


# Load and Merge Datasets

In [12]:
print("--" * 80)
print("LOADING DATASETS")
print("--" * 80)

# Load datasets
print(f"\nLoading {SMALL_PATH}...")
df_small = load_small_dataset(SMALL_PATH)
print(f"Loaded {len(df_small)} FAQ entries")

print(f"\nLoading {LARGE_PATH}...")
df_large = load_large_dataset(LARGE_PATH)
print(f"Loaded {len(df_large)} dialogue entries")

# Merge
print(f"\nMerging datasets...")
merged = pd.concat([df_small, df_large], ignore_index=True, sort=False)
print(f"Initial merged rows: {len(merged)}")

# DEDUPLICATION
print("\n" + "--" * 80)
print("DEDUPLICATION")
print("--" * 80)

# Create fingerprints for deduplication
merged['fp'] = merged.apply(
    lambda r: stable_fingerprint((r['prompt'] or '') + (r['response'] or '')),
    axis=1
)

before = len(merged)
merged = merged.drop_duplicates(subset='fp').reset_index(drop=True)
after = len(merged)
print(f"\nRemoved {before - after} duplicates")
print(f"Rows after dedup: {after}")

# CLEANING & FILTERING
print("\n" + "--" * 80)
print("CLEANING & FILTERING")
print("--" * 80)

merged = clean_and_filter_dataset(
    merged,
    min_char_length=MIN_CHAR_LENGTH,
    max_char_length=MAX_CHAR_LENGTH,
    remove_urls=REMOVE_URLS,
    verbose=True
)

# Ensure ID exists for all rows
def ensure_id(row):
    if row['id'] and str(row['id']).strip():
        return row['id']
    return f"u_{row['fp'][:12]}"

merged['id'] = merged.apply(ensure_id, axis=1)

# Keep only necessary columns
merged = merged[['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp']]

# Save cleaned English dataset
merged.to_csv(MERGED_EN_PATH, index=False)
print(f"\nSaved cleaned English dataset to: {MERGED_EN_PATH}")
print(f"  Final row count: {len(merged)}")

# Display sample
print("\nSample of cleaned data:")
print(merged[['prompt', 'response']].head(3))

STEP 1: LOADING DATASETS

Loading mentalhealth.csv...
  Loaded 97 FAQ entries

Loading train.csv...
  Loaded 3512 dialogue entries

Merging datasets...
  Initial merged rows: 3609

STEP 2: DEDUPLICATION

Removed 1043 duplicates
Rows after dedup: 2566

STEP 3: CLEANING & FILTERING
CLEANING DATASET
Initial rows: 2566
  → Converting to string and normalizing...
  → Filtering by length...

CLEANING COMPLETE!
Final rows: 1522
Removed: 1044 (40.7%)
Retained: 59.3% of original
Time: 0.27s

Saved cleaned English dataset to: output/merged_en.csv
  Final row count: 1522

Sample of cleaned data:
                                              prompt  \
0        What does it mean to have a mental illness?   
1                    Who does mental illness affect?   
2  What are some of the warning signs of mental i...   

                                            response  
0  Mental illnesses are health conditions that di...  
1  Mental illness does can affect anyone, regardl...  
2  Symptoms of men

# Translation Function

In [14]:
def deep_translate_one(text, target_lang):
    for attempt in range(MAX_RETRIES):
        try:
            return DeepGoogle(source='auto', target=target_lang).translate(text)
        except Exception as e:
            wait = 2 ** attempt
            if attempt < MAX_RETRIES - 1:
                print(f"Translation error: {e}. Retrying in {wait}s...")
                time.sleep(wait)
            else:
                print(f"Translation failed after {MAX_RETRIES} attempts. Returning original.")
                return text

def translate_batch(texts, target_lang):
    out = []
    for i in tqdm(range(len(texts)), desc=f"Translating to {target_lang}"):
        t = texts[i]
        out.append(deep_translate_one(t, target_lang))

        # Rate limiting
        if i % 5 == 0:
            time.sleep(SLEEP_BETWEEN_TRANSLATIONS)
        if i % 50 == 0:
            time.sleep(1)

    return out

def batch_translate_df(df, target_langs):
    rows = []
    n = len(df)

    for lang in target_langs:
        print(f"\n🌍 Translating to {lang.upper()}...")

        # Get all prompts and responses
        prompts = df['prompt'].tolist()
        responses = df['response'].tolist()

        # Translate
        translated_prompts = translate_batch(prompts, lang)
        translated_responses = translate_batch(responses, lang)

        # Create new rows with translations
        for i in range(n):
            base = df.iloc[i]
            rows.append({
                'id': base['id'],
                'source': base['source'],
                'type': base['type'],
                'prompt': translated_prompts[i],
                'response': translated_responses[i],
                'lang': lang,
                'orig_id': base['orig_id'],
                'fp': stable_fingerprint(translated_prompts[i] + translated_responses[i])
            })

        # Brief rest between languages
        time.sleep(2)

    # Also include original English rows
    en_rows = df.to_dict(orient='records')

    # Combine
    all_rows = en_rows + rows
    return pd.DataFrame(all_rows)

print("Translation functions defined")

Translation functions defined


Translate Dataset

In [15]:
print("\n" + "--" * 80)
print("TRANSLATION")
print("--" * 80)

print(f"\nTranslating dataset into: {TARGET_LANGS}")
print(f"This will take approximately {len(merged) * len(TARGET_LANGS) * 0.3 / 60:.1f} minutes")
print(f"(Translating {len(merged)} entries × {len(TARGET_LANGS)} languages)")

multilang = batch_translate_df(merged, TARGET_LANGS)

print(f"\nTranslation complete!")
print(f"Total rows (all languages): {len(multilang)}")
print(f"English: {len(multilang[multilang['lang'] == 'en'])}")
for lang in TARGET_LANGS:
    print(f"{lang.upper()}: {len(multilang[multilang['lang'] == lang])}")

# Save multilingual CSV
multilang.to_csv(MERGED_MULTILANG_PATH, index=False, encoding="utf-8-sig")
print(f"\nSaved multilingual dataset to: {MERGED_MULTILANG_PATH}")

# SAVE JSONL FORMAT (for fine-tuning)
print("\n" + "--" * 80)
print("SAVE JSONL FORMAT")
print("--" * 80)

with open(MERGED_JSONL_PATH, 'w', encoding='utf-8') as fh:
    for _, r in multilang.iterrows():
        obj = {
            'id': r['id'],
            'lang': r['lang'],
            'prompt': r['prompt'],
            'response': r['response']
        }
        fh.write(json.dumps(obj, ensure_ascii=False) + '\n')

print(f"\nSaved JSONL to: {MERGED_JSONL_PATH}")

# Display sample from each language
print("\nSample from each language:")
for lang in ['en'] + TARGET_LANGS:
    filtered = multilang[multilang['lang'] == lang]
    if len(filtered) == 0:
        print(f"\n[{lang.upper()}]\n  No samples found.")
        continue
    sample = filtered.iloc[0]
    print(f"\n[{lang.upper()}]")
    print(f"  Prompt:  {sample['prompt'][:80]}...")
    print(f"  Response: {sample['response'][:80]}...")

print("\n" + "--" * 80)
print("ALL PROCESSING COMPLETE!")
print("--" * 80)
print(f"\nOutput files created in '{OUT_DIR}':")
print(f"  1. {MERGED_EN_PATH.name} - English only")
print(f"  2. {MERGED_MULTILANG_PATH.name} - All languages (CSV)")
print(f"  3. {MERGED_JSONL_PATH.name} - All languages (JSONL)")


----------------------------------------------------------------------------------------------------------------------------------------------------------------
TRANSLATION
----------------------------------------------------------------------------------------------------------------------------------------------------------------

Translating dataset into: ['hi', 'te']
This will take approximately 15.2 minutes
(Translating 1522 entries × 2 languages)

🌍 Translating to HI...


Translating to hi: 100%|██████████| 1522/1522 [28:32<00:00,  1.13s/it]



🌍 Translating to TE...


Translating to te: 100%|██████████| 1522/1522 [14:06<00:00,  1.80it/s]



Translation complete!
Total rows (all languages): 4566
English: 1522
HI: 1522
TE: 1522

Saved multilingual dataset to: output/merged_multilang.csv

----------------------------------------------------------------------------------------------------------------------------------------------------------------
SAVE JSONL FORMAT
----------------------------------------------------------------------------------------------------------------------------------------------------------------

Saved JSONL to: output/merged_multilang.jsonl

Sample from each language:

[EN]
  Prompt:  What does it mean to have a mental illness?...
  Response: Mental illnesses are health conditions that disrupt a person's thoughts, emotion...

[HI]
  Prompt:  मानसिक बीमारी होने का क्या मतलब है?...
  Response: मानसिक बीमारियाँ स्वास्थ्य स्थितियाँ हैं जो किसी व्यक्ति के विचारों, भावनाओं, रि...

[TE]
  Prompt:  మానసిక అనారోగ్యం అంటే ఏమిటి?...
  Response: మానసిక అనారోగ్యాలు ఒక వ్యక్తి యొక్క ఆలోచనలు, భావోద్వేగాలు, సంబం

# Data Analysis

In [16]:
print("-" * 80)
print("DATA STATISTICS")
print("-" * 80)

# Overall statistics
print(f"\nOVERALL STATISTICS:")
print(f"Total rows: {len(multilang)}")
print(f"Unique prompts: {multilang['prompt'].nunique()}")
print(f"Unique responses: {multilang['response'].nunique()}")

# Language breakdown
print(f"\nLANGUAGE BREAKDOWN:")
lang_counts = multilang['lang'].value_counts()
for lang, count in lang_counts.items():
    pct = (count / len(multilang)) * 100
    print(f"{lang.upper()}: {count:,} rows ({pct:.1f}%)")

# Type breakdown (FAQ vs Dialogue)
print(f"\nTYPE BREAKDOWN:")
type_counts = multilang['type'].value_counts()
for typ, count in type_counts.items():
    pct = (count / len(multilang)) * 100
    print(f"{typ.upper()}: {count:,} rows ({pct:.1f}%)")

# Length statistics (English only)
en_data = multilang[multilang['lang'] == 'en']
prompt_lens = en_data['prompt'].str.len()
response_lens = en_data['response'].str.len()

print(f"\nLENGTH STATISTICS (English):")
print(f"Prompt length:")
print(f"Mean: {prompt_lens.mean():.1f} chars")
print(f"Median: {prompt_lens.median():.1f} chars")
print(f"Min: {prompt_lens.min()} chars")
print(f"Max: {prompt_lens.max()} chars")

print(f"Response length:")
print(f"Mean: {response_lens.mean():.1f} chars")
print(f"Median: {response_lens.median():.1f} chars")
print(f"Min: {response_lens.min()} chars")
print(f"Max: {response_lens.max()} chars")

--------------------------------------------------------------------------------
DATA STATISTICS
--------------------------------------------------------------------------------

OVERALL STATISTICS:
Total rows: 4566
Unique prompts: 2053
Unique responses: 3918

LANGUAGE BREAKDOWN:
EN: 1,522 rows (33.3%)
HI: 1,522 rows (33.3%)
TE: 1,522 rows (33.3%)

TYPE BREAKDOWN:
DIALOGUE: 4,431 rows (97.0%)
FAQ: 135 rows (3.0%)

LENGTH STATISTICS (English):
Prompt length:
Mean: 250.0 chars
Median: 221.0 chars
Min: 12 chars
Max: 992 chars
Response length:
Mean: 585.2 chars
Median: 587.0 chars
Min: 10 chars
Max: 999 chars


# Creating Train, val and test Splits

In [17]:
print("\n" + "-" * 80)
print("CREATING TRAIN/VAL/TEST SPLITS")
print("-" * 80)

def create_splits(df, lang='en', val_size=0.15, test_size=0.10):
    # Filter by language
    df_lang = df[df['lang'] == lang].copy()

    # First split: train+val vs test
    train_val, test = train_test_split(
        df_lang,
        test_size=test_size,
        stratify=df_lang['type'],
        random_state=42
    )

    # Second split: train vs val
    adjusted_val_size = val_size / (1 - test_size)
    train, val = train_test_split(
        train_val,
        test_size=adjusted_val_size,
        stratify=train_val['type'],
        random_state=42
    )

    return train, val, test

# Create splits for each language
for lang in ['en'] + TARGET_LANGS:
    print(f"\nCreating splits for {lang.upper()}...")

    train, val, test = create_splits(multilang, lang=lang)

    # Save splits
    train.to_csv(OUT_DIR / f"train_{lang}.csv", index=False)
    val.to_csv(OUT_DIR / f"val_{lang}.csv", index=False)
    test.to_csv(OUT_DIR / f"test_{lang}.csv", index=False)

    print(f"Train: {len(train):,} rows ({len(train)/len(multilang[multilang['lang']==lang])*100:.1f}%)")
    print(f"Val:   {len(val):,} rows ({len(val)/len(multilang[multilang['lang']==lang])*100:.1f}%)")
    print(f"Test:  {len(test):,} rows ({len(test)/len(multilang[multilang['lang']==lang])*100:.1f}%)")

print("\nAll splits created and saved")


--------------------------------------------------------------------------------
CREATING TRAIN/VAL/TEST SPLITS
--------------------------------------------------------------------------------

Creating splits for EN...
Train: 1,140 rows (74.9%)
Val:   229 rows (15.0%)
Test:  153 rows (10.1%)

Creating splits for HI...
Train: 1,140 rows (74.9%)
Val:   229 rows (15.0%)
Test:  153 rows (10.1%)

Creating splits for TE...
Train: 1,140 rows (74.9%)
Val:   229 rows (15.0%)
Test:  153 rows (10.1%)

All splits created and saved


Preparing for Model Training

In [18]:
print("\n" + "--" * 80)
print("PREPARING DATA FOR MODEL TRAINING")
print("--" * 80)

# Load English training data
train_df = pd.read_csv(OUT_DIR / "train_en.csv")
val_df = pd.read_csv(OUT_DIR / "val_en.csv")

# Extract inputs and targets
train_inputs = train_df["prompt"].tolist()
train_targets = train_df["response"].tolist()
val_inputs = val_df["prompt"].tolist()
val_targets = val_df["response"].tolist()

print(f"\nTraining data prepared:")
print(f"Training samples: {len(train_inputs):,}")
print(f"Validation samples: {len(val_inputs):,}")

# Sample data for verification
print(f"\nSample training examples:")
for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"Input:  {train_inputs[i][:100]}...")
    print(f"Target: {train_targets[i][:100]}...")

print("\n" + "**" * 80)
print("DATA PREPARATION COMPLETE")
print("**" * 80)


----------------------------------------------------------------------------------------------------------------------------------------------------------------
PREPARING DATA FOR MODEL TRAINING
----------------------------------------------------------------------------------------------------------------------------------------------------------------

Training data prepared:
Training samples: 1,140
Validation samples: 229

Sample training examples:

Example 1:
Input:  Is this something I should be worried about? Should I do something about it?...
Target: The answer depends on how the other areas of your son's life are doing.Is he happy or does he seem h...

Example 2:
Input:  I have an eating disorder of binging. I've had gastric sleeve surgery. I need help with issues of ab...
Target: It can be really frustrating to feel like your counselor is not providing you with the help you need...

Example 3:
Input:  There's this boy who asked me out awhile ago. I said no because my friends 